# Params
BATCH_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 1.0
OPTIM_GAMMA = 0.7
# Define NN
import torch
import torch.nn as nn
import torch.nn.functional as f



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 4, 1)
        self.conv2 = nn.Conv2d(32, 64, 4, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(28224, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = f.relu(x)
        x = self.conv2(x)
        x = f.relu(x)
        x = f.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = f.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        out = f.log_softmax(x, dim=1)
        return out



In [11]:
# Load the Data
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

train_dataset = datasets.ImageFolder(root = 'train', transform=transforms.Compose([transforms.Grayscale(num_output_channels=1), 
                                                         transforms.ToTensor()]))
test_dataset = datasets.ImageFolder(root = 'validation', transform=transforms.Compose([transforms.Grayscale(num_output_channels=1), 
                                                         transforms.ToTensor()]))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)



In [ ]:
# Display image and label.
import matplotlib.pyplot as plt

train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [13]:
# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

In [ ]:
# Test Feeding Model
train_features, train_labels = next(iter(train_loader))
train_features.to(device)
output = model(train_features)
print(f"Output = {output}")
print(f"Expected = {train_labels}")

In [ ]:
# Train Model
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

def train(model, train_loader, optimizer, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = f.nll_loss(output, target)
        loss.backward()
        optimizer.step()

def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += f.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

optimizer = optim.Adadelta(model.parameters(), lr=LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=1, gamma=OPTIM_GAMMA)

print(f'Training on: {device}')

print(torch.cuda.is_available())

print(torch.cuda.device_count())

for i in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, device)
    test(model, test_loader, device)
    scheduler.step()


